In [1]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

from geopy.geocoders import Nominatim

import requests
import lxml.html as lh
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries imported.')

Libraries imported.


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

In [3]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')

col=[]
i=0

for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighbourhood
"


In [5]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1


In [6]:
[len(C) for (title,C) in col]

[181, 181, 181]

In [7]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [8]:
df.head(5)

,Postal Code\n,Borough\n,Neighbourhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [9]:
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [10]:
df.head(2)

,Postcode,Borough,Neighbourhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n


In [11]:
df.columns = ['Postal Code','Borough','Neighbourhood']
df = df.replace('\n',' ', regex=True)
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [12]:
df.drop(df.index[df['Borough'] == 'Not assigned'], inplace = True)
df = df.reset_index(drop=True)

df.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [13]:
df = df.reset_index(drop=True)
df.head(15)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [14]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()
df.head(5)

,Postal Code,Borough,Neighbourhood
0,,Canadian postal codes,
1,M1A,Not assigned,Not assigned
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"


In [15]:
df = df.iloc[1:]
df.head()

,Postal Code,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M1B,Scarborough,"Malvern, Rouge"
3,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
4,M1E,Scarborough,"Guildwood, Morningside, West Hill"
5,M1G,Scarborough,Woburn


In [16]:
df = df.reset_index(drop=True)

In [17]:
df['Neighbourhood'] = df['Neighbourhood'].str.strip()
df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']

In [18]:
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M1B,Scarborough,"Malvern, Rouge"
2,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
3,M1E,Scarborough,"Guildwood, Morningside, West Hill"
4,M1G,Scarborough,Woburn
5,M1H,Scarborough,Cedarbrae
6,M1J,Scarborough,Scarborough Village
7,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
8,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
9,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"


In [19]:
df = df.sort_values(by='Borough')

In [20]:
df = df.reset_index(drop=True)
df

,Postal Code,Borough,Neighbourhood
0,M4S,Central Toronto,Davisville
1,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
2,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"
3,M5N,Central Toronto,Roselawn
4,M4N,Central Toronto,Lawrence Park
5,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
6,M4T,Central Toronto,"Moore Park, Summerhill East"
7,M4P,Central Toronto,Davisville North
8,M4R,Central Toronto,"North Toronto West, Lawrence Park"
9,M5W,Downtown Toronto,Stn A PO Boxes


In [21]:
df.drop(df.index[75:152],0,inplace=True)
df

,Postal Code,Borough,Neighbourhood
0,M4S,Central Toronto,Davisville
1,M5R,Central Toronto,"The Annex, North Midtown, Yorkville"
2,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park"
3,M5N,Central Toronto,Roselawn
4,M4N,Central Toronto,Lawrence Park
5,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest..."
6,M4T,Central Toronto,"Moore Park, Summerhill East"
7,M4P,Central Toronto,Davisville North
8,M4R,Central Toronto,"North Toronto West, Lawrence Park"
9,M5W,Downtown Toronto,Stn A PO Boxes


In [22]:
df = df.sort_values(by='Postal Code')
df = df.reset_index(drop=True)

df

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [23]:
df.to_csv(r'df_can.csv')